In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc


CPU times: user 540 ms, sys: 2.2 s, total: 2.74 s
Wall time: 460 ms


/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['DOB'] = df['DOB'].fillna('11/01/82')

df['Lead_Creation_Date_ori'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date_ori'] = test['Lead_Creation_Date'].apply(helper)

df['DOB_ori'] = df['DOB'].apply(helper)
test['DOB_ori'] = test['DOB'].apply(helper)

def extract_date(d):
    tmp = strptime(d, '%d/%m/%y')
    y = tmp.tm_year
    m = tmp.tm_mon
    dy = tmp.tm_mday
    w = tmp.tm_wday
    yd = tmp.tm_yday
    dst = tmp.tm_isdst
    return '/'.join(map(str,[y,m,dy,w,yd,dst]))

df['DOB_e'] = df['DOB'].apply(extract_date)
df['Lead_Creation_Date_e'] = df['Lead_Creation_Date'].apply(extract_date)

for i in range(6):
    df['DOB_e_'+str(i)] = df['DOB_e'].apply(lambda x: int(x.split('/')[i]))
    df['Lead_Creation_Date_e'+str(i)] = df['Lead_Creation_Date_e'].apply(lambda x: int(x.split('/')[i]))

_ = df.pop('DOB_e')
_ = df.pop('Lead_Creation_Date_e')
_ = df.pop('DOB')
_ = df.pop('Lead_Creation_Date')

(69713, 22)
(30037, 21)
CPU times: user 4.09 s, sys: 147 ms, total: 4.24 s
Wall time: 4.23 s


### fill null

In [27]:
%%time
df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

s = StandardScaler()
df = s.fit_transform(df)
test = s.transform(test)

CPU times: user 54.2 s, sys: 36.9 s, total: 1min 31s
Wall time: 1min 31s


In [28]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        77G        48G       1.4M        66M        81M
-/+ buffers/cache:        77G        48G 
Swap:         4.0G       192M       3.8G 


### train test split

In [29]:
%%time
m = lambda *x: [i.shape for i in x]
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state = 7)
print m(x_train, y_train, x_test, y_test)


[(55770, 37428), (55770,), (13943, 37428), (13943,)]
CPU times: user 27.4 s, sys: 12.2 s, total: 39.5 s
Wall time: 39.5 s


In [35]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        77G        48G       1.4M        66M        81M
-/+ buffers/cache:        77G        48G 
Swap:         4.0G       192M       3.8G 


In [33]:
import gc
del df
gc.collect()

90

In [34]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        77G        48G       1.4M        66M        81M
-/+ buffers/cache:        77G        48G 
Swap:         4.0G       192M       3.8G 


### create xgb matrix

In [36]:
%%time
d_train = xgb.DMatrix(x_train, label=y_train)
d_test = xgb.DMatrix(x_test, label=y_test)


CPU times: user 19.9 s, sys: 8.98 s, total: 28.9 s
Wall time: 28.9 s


In [40]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        77G        48G       1.4M        66M        81M
-/+ buffers/cache:        77G        48G 
Swap:         4.0G       192M       3.8G 


In [39]:
del x_train, x_test
gc.collect()

0

In [41]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        77G        48G       1.4M        66M        81M
-/+ buffers/cache:        77G        48G 
Swap:         4.0G       192M       3.8G 


In [42]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.05,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [44]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=20, 
                verbose_eval=1,  maximize=True,
               )
                

'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.746783	test-auc:0.739669
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 20 rounds.
[1]	train-auc:0.775006	test-auc:0.764923
[2]	train-auc:0.787902	test-auc:0.773816
[3]	train-auc:0.793864	test-auc:0.774534
[4]	train-auc:0.824076	test-auc:0.796327
[5]	train-auc:0.82686	test-auc:0.795537
[6]	train-auc:0.828731	test-auc:0.794946
[7]	train-auc:0.841846	test-auc:0.795495
[8]	train-auc:0.847009	test-auc:0.794016
[9]	train-auc:0.851285	test-auc:0.794811
[10]	train-auc:0.855302	test-auc:0.796215
[11]	train-auc:0.868296	test-auc:0.804614
[12]	train-auc:0.870605	test-auc:0.806003
[13]	train-auc:0.872105	test-auc:0.806224
[14]	train-auc:0.874093	test-auc:0.806339
[15]	train-auc:0.876361	test-auc:0.80768
[16]	train-auc:0.877557	test-auc:0.806639
[17]	train-auc:0.880151	test-auc:0.807109
[18]	train-auc:0.882914	test-auc:0.807296
[19]	train-auc:0.884704	test-auc:0.807072
[20]	train-auc:0.886671	test-au

'!!! used 858.95 s'

In [66]:
clf.save_rabit_checkpoint?

In [45]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 13.9 s, sys: 6.85 s, total: 20.7 s
Wall time: 20.7 s


In [46]:
%%time
predb = clf.predict(dtest)


CPU times: user 4.31 s, sys: 3 ms, total: 4.31 s
Wall time: 268 ms


___

In [47]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [48]:
submit.to_csv('xgb-ext-date.csv',index=False)

In [22]:
clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba